# extract FITS image  

In [1]:
from lsst.daf.butler import Butler
from lsst.daf.butler.registry import Registry


In [15]:
repo = 'embargo'
instrument = 'LSSTComCam'
datasetType = 'raw'
# raw all collection contains all images 
collections = ["LSSTComCam/defaults"]

butler = Butler(repo, collections=collections, instrument=instrument)

registry = butler.registry

#uncomment to get the help page
#help(registry) 

## Find the exposures
Listing the images could be done with butler using one of the tutorial queries.
The GEO observatons were taken on 2024 Dec 17, day_obs = 20241107, BLOCK-T332.
Keith mentioned there may be some other observations mixed in.

In [16]:
day = 20241207
block = 'BLOCK-T332'
dims = ['exposure']
result = registry.queryDataIds(  datasets=datasetType, dimensions = dims,
          where = f"day_obs = {day}  AND exposure.science_program='{block}'"
                              )
results = [r for r in result]

print (f"got {len(results)} images")

got 270 images


ComCam has 9 images per exposure - this will list the exposures. 
The last digits after the daynumber  match the 'Seq. No.' in RubinTV

In [17]:
exp=""
exposures = []
for r in results:
   if r['exposure'] != exp:
        exp=  r['exposure']
        exposures.append(exp)

print ("there are {len(exposures)} exposures")
print (exposures)


there are {len(exposures)} exposures
[2024120700539, 2024120700540, 2024120700541, 2024120700542, 2024120700543, 2024120700544, 2024120700545, 2024120700546, 2024120700547, 2024120700548, 2024120700549, 2024120700550, 2024120700560, 2024120700551, 2024120700552, 2024120700553, 2024120700554, 2024120700555, 2024120700556, 2024120700557, 2024120700558, 2024120700559, 2024120700561, 2024120700562, 2024120700563, 2024120700564, 2024120700565, 2024120700566, 2024120700567, 2024120700568]


This is just getting the datasetref  the first exposure (this is comcam so that shoudl be 9 images one per detector)

In [18]:
datasetRefs = registry.queryDatasets(datasetType=datasetType,
                                     where=f'exposure = {exposures[3]}')


## Write fits 
Write out fits files for the first count images. 
By default you will not have enough space to write all the files.

In [19]:
images = []
for count, exp in enumerate(datasetRefs):
    fn=f"Rubin-{exp.dataId['exposure']}-{exp.dataId['detector']}.fits"
    img = butler.get(datasetType,exp.dataId)
    img.writeFits(fn)
    images.append(exp)
    
print (f"{count+1} images output")

9 images output


Now we have Fits files we can use with external code.
We get the list of files we made and process them with SEP (Source Extractor Python) as an example of external code. 
We will make a list of Objects and store that.

In [ ]:
import glob
filelist = glob.glob('Rubin-*.fits')
print (filelist)

## If you want to plot one of these 

In [ ]:
import pylab as plt
import lsst.afw.display as afwDisplay

afwDisplay.setDefaultBackend('matplotlib')

for exp in datasetRefs:
    dataId=exp.dataId
    img = butler.get('raw', dataId=dataId)

    fig = plt.figure()
    display = afwDisplay.Display()
    display.scale('linear', 'zscale')
    display.mtv(img)
    plt.show()